# nbstata: a new Stata kernel

> A Jupyter kernel for Stata built on pystata

&nbsp;

*nbstata* is a [Jupyter kernel](https://docs.jupyter.org/en/latest/projects/kernels.html) for [Stata](https://www.stata.com/why-use-stata/) built on top of [pystata](https://www.stata.com/python/pystata18/index.html).

**[*For the User Guide, click here.*](https://hugetim.github.io/nbstata/user_guide.html)**

&nbsp;

## What is Jupyter?

A Jupyter notebook allows you to combine interactive code and results with [Markdown](https://daringfireball.net/projects/markdown/basics) in a single document. Though it is named after the three core programming languages it supports (Julia, Python, and R), it can be used with with a wide variety of languages. 

*nbstata* allows you to create Stata notebooks (as opposed to [using Stata within a *Python* notebook](https://www.stata.com/python/pystata18/notebook/Example2.html), which is needlessly clunky if you are working primarily with Stata).

### Key *nbstata* features

- [x] [Easy setup](https://hugetim.github.io/nbstata/user_guide.html#install)
- [x] Works with Stata 17+ (only).
- [x] DataGrid widget with `browse`-like capabilities (e.g., interactive filtering)
- [x] Variable and data properties available in a 'contextual help' side panel
- [x] Quarto [inline code](https://quarto.org/docs/computations/inline-code.html) support

Users of Stata 17 or 18.0 also get these features only built-in natively to Stata 18.5+:

- Displays Stata output without the redundant 'echo' of (multi-line) commands
- Autocompletion for variables, macros, matrices, and file paths
- Interactive/richtext help files accessible within notebook
- `#delimit ;` interactive support (along with all types of comments)

The video below demonstrates using Stata in a Jupyter notebook. In addition to the [NBClassic](https://nbclassic.readthedocs.io/en/stable/notebook.html) application shown there, *nbstata* can also be used with [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html), [VS Code](https://code.visualstudio.com/docs/datascience/jupyter-notebooks), or [Quarto](https://quarto.org/).

<img align="center" width="650" src="https://github.com/kylebarron/stata_kernel/raw/master/docs/src/img/jupyter_notebook_example.gif" alt="Animated GIF demoing major Stata kernel features">

### What can you do with Stata notebooks...
...that you can't do with the [official Stata interface](https://www.stata.com/features/overview/graphical-user-interface/)?

* Exploratory analysis that is both:
    * interactive
    * preserved for future reference/editing

* Present results in a way that interweaves:[^1]
    * code
    * results (including graphs)
    * rich text: 
        1. lists
        2. **Headings**
        3. <img align="left" width="54" height="18.6" src="https://raw.githubusercontent.com/hugetim/nbstata/master/index_files/figure-commonmark/226326ec-1-image-2.png" alt="WordArt of the word 'images'">
        4. [links](https://hugetim.github.io/nbstata/) 
        5. math: $y_{it}=\beta_0+\varepsilon_{it}$

[^1]: Stata [dynamic documents](https://www.stata.com/manuals/rptdynamicdocumentsintro.pdf) can do this part, though with a less interactive workflow. (See also: [markstat](https://grodri.github.io/markstat/), [stmd](https://www.ssc.wisc.edu/~hemken/Stataworkshops/stmd/Usage/stmdusage.html), and [Statamarkdown](https://ssc.wisc.edu/~hemken/Stataworkshops/Statamarkdown/stata-and-r-markdown.html)) Using *nbstata* with [Quarto](https://www.statalist.org/forums/forum/general-stata-discussion/general/1703835-ado-files-and-literate-programming) instead gives you a similar workflow, with greater flexibility of output.

## Contributing

*nbstata* is being developed using [nbdev](https://nbdev.fast.ai/blog/posts/2022-07-28-nbdev2/#whats-nbdev). The `/nbs` directory is where edits to the source code should be made. (The python code is then exported to the `/nbdev` library folder.)

For more, see [CONTRIBUTING.md](https://github.com/hugetim/nbstata/blob/master/CONTRIBUTING.md).

## Acknowledgements

Kyle Barron authored the original *stata_kernel*, which works for older versions of Stata. Vinci Chow created a Stata kernel that instead uses [pystata](https://www.stata.com/python/pystata18/), which first became available with Stata 17. *nbstata* was originally derived from his [*pystata-kernel*](https://github.com/ticoneva/pystata-kernel), but much of the docs and newer features are derived from *stata_kernel*.